# Ficore AI - Financial Health Score Calculation

This notebook is used for developing and testing the logic for the Ficore AI Financial Health Score application. It includes data retrieval from Google Sheets, score calculation, ranking, and email sending functionality. The final implementation will be deployed in a Flask app on Render.

## Steps:
1. **Authenticate with Google Sheets API** to fetch user data.
2. **Calculate Financial Health Scores** based on income, expenses, debt, and interest rates.
3. **Rank Users** based on their health scores.
4. **Send Emails** to users with their scores, breakdowns, and advice.
5. **Store Results** in a separate Google Sheet tab for record-keeping.

## Step 1: Import Libraries and Set Up Environment

We need to import the necessary libraries and load environment variables for authentication.

In [ ]:
import os
import pandas as pd
import smtplib
import time
import re
import json
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Constants for Google Sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '13hbiMTMRBHo9MHjWwcugngY_aSiuxII67HCf03MiZ8I'
DATA_RANGE_NAME = 'Sheet1!A1:L'
RESULTS_SHEET_NAME = 'FicoreAIResults'
RESULTS_HEADER = ['Email', 'FicoreAIScore', 'FicoreAIRank']
FEEDBACK_FORM_URL = 'https://forms.gle/ficoreai-feedback'

# Predetermined headers for Sheet1 (in Google Form order, including AutoEmail)
PREDETERMINED_HEADERS = [
    'Timestamp', 'BusinessName', 'IncomeRevenue', 'ExpensesCosts', 'DebtLoan',
    'DebtInterestRate', 'AutoEmail', 'PhoneNumber', 'FirstName', 'LastName', 'UserType', 'Email'
]

## Step 2: Authenticate with Google Sheets API

We'll authenticate using the service account credentials stored in the environment variable.

In [ ]:
def authenticate_google_sheets():
    """Authenticate with Google Sheets API using Service Account credentials from environment variable."""
    creds_json = os.environ.get('GOOGLE_CREDENTIALS_JSON')
    if not creds_json:
        raise Exception("GOOGLE_CREDENTIALS_JSON environment variable not set.")
    try:
        creds_info = json.loads(creds_json)
        creds = service_account.Credentials.from_service_account_info(creds_info, scopes=SCOPES)
        return build('sheets', 'v4', credentials=creds)
    except json.JSONDecodeError as e:
        raise Exception(f"Error decoding GOOGLE_CREDENTIALS_JSON: {e}")
    except Exception as e:
        raise Exception(f"Error authenticating with Google Sheets: {e}")

# Test authentication
service = authenticate_google_sheets()
print("Google Sheets API authentication successful.")

## Step 3: Fetch Data from Google Sheet

We'll fetch the data from the Google Sheet and load it into a Pandas DataFrame.

In [ ]:
def fetch_data_from_sheet():
    """Fetch data from Sheet1 in the Google Sheet."""
    try:
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=DATA_RANGE_NAME).execute()
        values = result.get('values', [])
        if not values:
            return None
        return values
    except Exception as e:
        raise Exception(f"Failed to fetch data from Google Sheet: {str(e)}")

# Fetch the data
sheet_data = fetch_data_from_sheet()
if sheet_data:
    headers = sheet_data[0]
    rows = sheet_data[1:]
    df = pd.DataFrame(rows, columns=headers)
    print("Data fetched successfully:")
    display(df)
else:
    print("No data found in the sheet.")
    df = pd.DataFrame(columns=PREDETERMINED_HEADERS)

## Step 4: Calculate Financial Health Score

We'll calculate the Financial Health Score based on the following metrics:
- **Cash Flow Ratio**: (Income - Expenses) / Income
- **Debt-to-Income Ratio**: Debt / Income
- **Debt Interest Burden**: Interest Rate / 20 (normalized to 0-1 scale)

Each metric is normalized to a 0-1 scale, weighted equally (1/3 each), and then scaled to a 0-100 score.

In [ ]:
def calculate_health_score(df):
    """Calculate the Financial Health Score based on income, expenses, debt, and interest rate."""
    try:
        # Convert columns to numeric, handle errors
        numeric_cols = ['IncomeRevenue', 'ExpensesCosts', 'DebtLoan', 'DebtInterestRate']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

        df['HealthScore'] = 0.0
        # Avoid division by zero by replacing 0 income with a small value
        df['IncomeRevenueSafe'] = df['IncomeRevenue'].replace(0, 1e-10)
        # Cash Flow Ratio
        df['CashFlowRatio'] = (df['IncomeRevenue'] - df['ExpensesCosts']) / df['IncomeRevenueSafe']
        # Debt-to-Income Ratio
        df['DebtToIncomeRatio'] = df['DebtLoan'] / df['IncomeRevenueSafe']
        # Debt Interest Burden (normalized by dividing by 20, assuming 20% is a high interest rate)
        df['DebtInterestBurden'] = df['DebtInterestRate'].clip(lower=0) / 20
        df['DebtInterestBurden'] = df['DebtInterestBurden'].clip(upper=1)
        # Normalize metrics to 0-1 scale
        df['NormCashFlow'] = df['CashFlowRatio'].clip(0, 1)
        df['NormDebtToIncome'] = 1 - df['DebtToIncomeRatio'].clip(0, 1)
        df['NormDebtInterest'] = 1 - df['DebtInterestBurden']
        # Calculate final score (equal weights, scale to 100)
        df['HealthScore'] = (df['NormCashFlow'] * 0.333 +
                            df['NormDebtToIncome'] * 0.333 +
                            df['NormDebtInterest'] * 0.333) * 100
        df['HealthScore'] = df['HealthScore'].round(2)

        # Provide advice based on the score
        def score_description(row):
            score = row['HealthScore']
            cash_flow = row['CashFlowRatio']
            debt_to_income = row['DebtToIncomeRatio']
            debt_interest = row['DebtInterestBurden']
            if score >= 75:
                return 'Stable; invest excess now'
            elif score >= 50:
                if cash_flow < 0.3 or debt_interest > 0.5:
                    return 'At Risk; manage expense'
                return 'Moderate; save monthly'
            elif score >= 25:
                if debt_to_income > 0.5 or debt_interest > 0.5:
                    return 'At Risk; pay off debt, manage expense'
                return 'At Risk; manage expense'
            else:
                if debt_to_income > 0.5 or cash_flow < 0.3:
                    return 'Critical; add source of income, pay off debt, manage expense'
                return 'Critical; seek financial help'

        df['ScoreDescription'] = df.apply(score_description, axis=1)
        return df
    except Exception as e:
        raise Exception(f"Failed to calculate HealthScore: {str(e)}")

# Calculate scores for the fetched data
if not df.empty:
    df = calculate_health_score(df)
    print("Health Scores calculated:")
    display(df[['Email', 'HealthScore', 'ScoreDescription']])
else:
    print("No data to calculate scores for.")

## Step 5: Rank Users

We'll sort users by their Health Scores and assign ranks.

In [ ]:
if not df.empty:
    # Sort by HealthScore (descending) and assign ranks
    df = df.sort_values(by='HealthScore', ascending=False)
    df['Rank'] = range(1, len(df) + 1)
    print("Rankings assigned:")
    display(df[['Email', 'HealthScore', 'Rank']])
else:
    print("No data to rank.")

## Step 6: Send Emails to Users

We'll send an email to each user with their Financial Health Score, breakdown, and advice. We'll use the user-typed Email as the primary address and AutoEmail as a fallback if the primary fails.

In [ ]:
def send_email(primary_email, fallback_email, first_name, last_name, rank, timestamp, health_score, score_description, cash_flow_ratio, debt_to_income_ratio, debt_interest_burden):
    """Send an email with the Financial Health Score, breakdown, and advice to the user, with a fallback email option."""
    max_retries = 3
    retry_delay = 5
    full_name = f"{first_name} {last_name}".strip()

    # Format the breakdown metrics as percentages
    cash_flow_score = round(cash_flow_ratio * 100, 2)
    debt_to_income_score = round((1 - debt_to_income_ratio) * 100, 2)
    debt_interest_score = round((1 - debt_interest_burden) * 100, 2)

    # Determine status for each metric
    def get_status(score):
        if score >= 75:
            return "Excellent"
        elif score >= 50:
            return "Good"
        elif score >= 25:
            return "Needs Attention"
        else:
            return "Critical"

    cash_flow_status = get_status(cash_flow_score)
    debt_to_income_status = get_status(debt_to_income_score)
    debt_interest_status = get_status(debt_interest_score)

    # Email content (same for both attempts)
    sender_email = os.environ.get('SENDER_EMAIL')
    sender_password = os.environ.get('SENDER_PASSWORD')
    if not sender_email or not sender_password:
        raise Exception("SENDER_EMAIL or SENDER_PASSWORD environment variables not set.")

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['Subject'] = f"Ficore AI: Your Financial Health Score, {full_name}"
    html = (
        '<html>\n'
        '    <body style="font-family: Arial, sans-serif; color: #333;">\n'
        '        <div style="text-align: center;">\n'
        '            <img src="https://www.freepik.com/free-photos-vectors/personal-finance-logo" alt="Ficore AI Logo" style="width: 150px; margin-bottom: 20px;" />\n'
        '        </div>\n'
        '        <h2 style="color: #2c3e50; text-align: center;">Ficore AI Financial Health Score</h2>\n'
        f'        <p>Hi {first_name},</p>\n'
        '        <p>We’re excited to share your Ficore AI Financial Health Score! This score reflects your financial strength based on three key factors: your cash flow, debt-to-income ratio, and debt interest burden. Let’s break it down for you.</p>\n'
        '        <h3 style="color: #2c3e50;">Your Financial Health Overview</h3>\n'
        '        <table style="border-collapse: collapse; width: 100%; max-width: 600px; margin: 20px 0;">\n'
        '            <tr style="background-color: #2c3e50; color: white;">\n'
        '                <th style="border: 1px solid #ddd; padding: 8px;">Rank</th>\n'
        '                <th style="border: 1px solid #ddd; padding: 8px;">Timestamp</th>\n'
        '                <th style="border: 1px solid #ddd; padding: 8px;">Name</th>\n'
        '                <th style="border: 1px solid #ddd; padding: 8px;">Health Score</th>\n'
        '                <th style="border: 1px solid #ddd; padding: 8px;">Advice</th>\n'
        '            </tr>\n'
        '            <tr>\n'
        f'                <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">{rank}</td>\n'
        f'                <td style="border: 1px solid #ddd; padding: 8px;">{timestamp}</td>\n'
        f'                <td style="border: 1px solid #ddd; padding: 8px;">{full_name}</td>\n'
        f'                <td style="border: 1px solid #ddd; padding: 8px; text-align: center;">{health_score}</td>\n'
        f'                <td style="border: 1px solid #ddd; padding: 8px;">{score_description}</td>\n'
        '            </tr>\n'
        '        </table>\n'
        '        <h3 style="color: #2c3e50;">How We Calculated Your Score</h3>\n'
        '        <p>Your Financial Health Score is a combination of three factors, each contributing equally to your overall score (out of 100). Here’s how you performed in each area:</p>\n'
        '        <ul>\n'
        f'            <li><strong>Cash Flow ({cash_flow_score}% - {cash_flow_status}):</strong> This measures how much money you have left after expenses. A higher percentage means you’re managing your expenses well relative to your income.</li>\n'
        f'            <li><strong>Debt-to-Income Ratio ({debt_to_income_score}% - {debt_to_income_status}):</strong> This shows how much of your income goes toward debt. A higher percentage indicates you have less debt relative to your income, which is a good sign.</li>\n'
        f'            <li><strong>Debt Interest Burden ({debt_interest_score}% - {debt_interest_status}):</strong> This reflects the impact of interest rates on your debt. A higher percentage means your debt interest rates are manageable.</li>\n'
        '        </ul>\n'
        f'        <p>{first_name}, your score of {health_score} is a great starting point! Follow the advice above to improve your financial health. We’re here to support you every step of the way—take one small action today to grow stronger financially for your business, your goals, and your future.</p>\n'
        '        <div style="text-align: center; margin: 20px 0;">\n'
        '            <a href="https://forms.gle/ficoreai-feedback" style="background-color: #007bff; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; display: inline-block; margin-right: 10px;">Help us improve! Share your feedback (takes 1 min)</a>\n'
        '            <a href="https://calendly.com/ficoreai" style="background-color: #28a745; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; display: inline-block;">Book Consultation</a>\n'
        '        </div>\n'
        f'        <p>Best regards,<br>Hassan<br>Ficore AI - Empowering African Financial Growth<br>Email: {sender_email} | Website: ficore.com.ng (coming soon)</p>\n'
        '    </body>\n'
        '</html>'
    )
    msg.attach(MIMEText(html, 'html'))

    # Try sending to the primary email first
    email_addresses = [(primary_email, "primary email"), (fallback_email, "fallback email")]
    email_sent = False

    for email_address, email_type in email_addresses:
        if not email_address:
            continue
        for attempt in range(max_retries):
            try:
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.starttls()
                server.login(sender_email, sender_password)
                msg['To'] = email_address
                server.send_message(msg)
                server.quit()
                time.sleep(1)
                print(f"Email successfully sent to {email_address} ({email_type})")
                email_sent = True
                break  # Exit retry loop on success
            except Exception as e:
                if attempt < max_retries - 1:
                    print(f"Attempt {attempt + 1} failed for {email_address} ({email_type}): {str(e)}. Retrying...")
                    time.sleep(retry_delay)
                else:
                    print(f"Failed to send email to {email_address} ({email_type}) after {max_retries} attempts: {str(e)}")
        if email_sent:
            break  # Exit email address loop if email was sent successfully

    if not email_sent:
        raise Exception(f"Failed to send email to both {primary_email} (primary) and {fallback_email} (fallback) after all attempts.")

# Send emails to all users (for testing, you might want to limit to a subset)
if not df.empty:
    for index, row in df.iterrows():
        try:
            send_email(
                primary_email=row['Email'],
                fallback_email=row['AutoEmail'],
                first_name=row['FirstName'],
                last_name=row['LastName'],
                rank=row['Rank'],
                timestamp=row['Timestamp'],
                health_score=row['HealthScore'],
                score_description=row['ScoreDescription'],
                cash_flow_ratio=row['NormCashFlow'],
                debt_to_income_ratio=row['DebtToIncomeRatio'],
                debt_interest_burden=row['DebtInterestBurden']
            )
        except Exception as e:
            print(f"Failed to send email to {row['Email']}: {str(e)}")
else:
    print("No users to send emails to.")

## Step 7: Store Results in Google Sheet

We'll store the rankings in a separate sheet called `FicoreAIResults`.

In [ ]:
def create_results_sheet():
    """Create the FicoreAIResults sheet if it doesn't exist and set up headers."""
    try:
        spreadsheet = service.spreadsheets()
        sheets = spreadsheet.get(spreadsheetId=SPREADSHEET_ID).execute().get('sheets', [])
        sheet_names = [sheet['properties']['title'] for sheet in sheets]
        if RESULTS_SHEET_NAME not in sheet_names:
            body = {'properties': {'title': RESULTS_SHEET_NAME}}
            spreadsheet.batchUpdate(spreadsheetId=SPREADSHEET_ID, body={'requests': [{'addSheet': body}]}).execute()
            header_range = f'{RESULTS_SHEET_NAME}!A1:C1'
            body = {'values': [RESULTS_HEADER]}
            spreadsheet.values().update(spreadsheetId=SPREADSHEET_ID, range=header_range, valueInputOption='USER_ENTERED', body=body).execute()
    except Exception as e:
        raise Exception(f"Failed to create or set up results sheet: {str(e)}")

def write_results_to_sheet(df):
    """Write Health Scores and Ranks to the FicoreAIResults sheet."""
    try:
        spreadsheet = service.spreadsheets()
        clear_range = f'{RESULTS_SHEET_NAME}!A2:C'
        spreadsheet.values().clear(spreadsheetId=SPREADSHEET_ID, range=clear_range).execute()
        results_data = []
        for index, row in df[['Email', 'HealthScore', 'Rank']].iterrows():
            results_data.append([row['Email'], float(row['HealthScore']), int(row['Rank'])])
        if results_data:
            body = {'values': results_data}
            range_name = f'{RESULTS_SHEET_NAME}!A2'
            spreadsheet.values().update(spreadsheetId=SPREADSHEET_ID, range=range_name,
                                        valueInputOption='USER_ENTERED', body=body).execute()
    except Exception as e:
        raise Exception(f"Failed to write results to Google Sheet ({RESULTS_SHEET_NAME}): {str(e)}")

# Create the results sheet and write data
if not df.empty:
    create_results_sheet()
    write_results_to_sheet(df)
    print("Results written to FicoreAIResults sheet.")
else:
    print("No data to write to results sheet.")

## Step 8: Simulate Form Submission (for Testing)

To simulate a form submission, we'll append a new row to the sheet, calculate the score, rank the user, send an email, and update the results sheet.

In [ ]:
def append_to_sheet(data):
    """Append form submission data to Sheet1 and apply formatting."""
    try:
        # Ensure the headers are set to the predetermined values before appending data
        header_range = 'Sheet1!A1:L1'
        body = {'values': [PREDETERMINED_HEADERS]}
        service.spreadsheets().values().update(
            spreadsheetId=SPREADSHEET_ID,
            range=header_range,
            valueInputOption='USER_ENTERED',
            body=body
        ).execute()

        range_name = 'Sheet1!A:L'
        # Append the data
        body = {'values': [data]}
        result = service.spreadsheets().values().append(spreadsheetId=SPREADSHEET_ID, range=range_name,
                                                        valueInputOption='USER_ENTERED', body=body).execute()

        # Get the row number of the newly added row
        sheet_metadata = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
        sheet = sheet_metadata['sheets'][0]  # Sheet1 is the first sheet
        row_count = sheet['properties']['gridProperties']['rowCount']
        new_row_index = row_count - 1  # The new row is the last row

        # Formatting requests
        requests = [
            # Set column widths
            {
                "updateDimensionProperties": {
                    "range": {
                        "sheetId": sheet['properties']['sheetId'],
                        "dimension": "COLUMNS",
                        "startIndex": 0,  # Column A
                        "endIndex": 1
                    },
                    "properties": {"pixelSize": 150},  # Adjust width for Timestamp
                    "fields": "pixelSize"
                }
            },
            {
                "updateDimensionProperties": {
                    "range": {
                        "sheetId": sheet['properties']['sheetId'],
                        "dimension": "COLUMNS",
                        "startIndex": 1,  # Column B
                        "endIndex": 2
                    },
                    "properties": {"pixelSize": 120},  # Adjust width for Business Name
                    "fields": "pixelSize"
                }
            },
            # Apply currency formatting to IncomeRevenue, ExpensesCosts, DebtLoan
            {
                "repeatCell": {
                    "range": {
                        "sheetId": sheet['properties']['sheetId'],
                        "startRowIndex": new_row_index,
                        "endRowIndex": new_row_index + 1,
                        "startColumnIndex": 2,  # Column C (IncomeRevenue)
                        "endColumnIndex": 5      # Column F (DebtLoan)
                    },
                    "cell": {
                        "userEnteredFormat": {
                            "numberFormat": {
                                "type": "CURRENCY",
                                "pattern": "₦#,##0.00"  # Use Naira symbol
                            },
                            "horizontalAlignment": "RIGHT"
                        }
                    },
                    "fields": "userEnteredFormat(numberFormat,horizontalAlignment)"
                }
            },
            # Apply number formatting to DebtInterestRate
            {
                "repeatCell": {
                    "range": {
                        "sheetId": sheet['properties']['sheetId'],
                        "startRowIndex": new_row_index,
                        "endRowIndex": new_row_index + 1,
                        "startColumnIndex": 5,  # Column F (DebtInterestRate)
                        "endColumnIndex": 6
                    },
                    "cell": {
                        "userEnteredFormat": {
                            "numberFormat": {
                                "type": "NUMBER",
                                "pattern": "0.00"
                            },
                            "horizontalAlignment": "RIGHT"
                        }
                    },
                    "fields": "userEnteredFormat(numberFormat,horizontalAlignment)"
                }
            },
            # Ensure text alignment for other columns (left-aligned)
            {
                "repeatCell": {
                    "range": {
                        "sheetId": sheet['properties']['sheetId'],
                        "startRowIndex": new_row_index,
                        "endRowIndex": new_row_index + 1,
                        "startColumnIndex": 0,  # Column A (Timestamp)
                        "endIndex": 2      # Column B (Business Name)
                    },
                    "cell": {
                        "userEnteredFormat": {
                            "horizontalAlignment": "LEFT"
                        }
                    },
                    "fields": "userEnteredFormat(horizontalAlignment)"
                }
            },
            {
                "repeatCell": {
                    "range": {
                        "sheetId": sheet['properties']['sheetId'],
                        "startRowIndex": new_row_index,
                        "endRowIndex": new_row_index + 1,
                        "startColumnIndex": 6,  # Column G (AutoEmail)
                        "endColumnIndex": 12     # Column L (Email)
                    },
                    "cell": {
                        "userEnteredFormat": {
                            "horizontalAlignment": "LEFT"
                        }
                    },
                    "fields": "userEnteredFormat(horizontalAlignment)"
                }
            }
        ]
        # Apply formatting
        service.spreadsheets().batchUpdate(spreadsheetId=SPREADSHEET_ID, body={'requests': requests}).execute()
    except Exception as e:
        raise Exception(f"Failed to append data to Google Sheet: {str(e)}")

# Simulate a form submission
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
test_data = [
    timestamp,              # Timestamp
    'Test Business',        # BusinessName
    1000000,                # IncomeRevenue
    500000,                 # ExpensesCosts
    200000,                 # DebtLoan
    5,                      # DebtInterestRate
    'test.primary@example.com',  # AutoEmail
    '1234567890',           # PhoneNumber
    'Test',                 # FirstName
    'User',                 # LastName
    'SME',                  # UserType
    'test.fallback@example.com'  # Email
]

# Validate primary email
email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
if not re.match(email_pattern, test_data[11]):  # test_data[11] is the Email field
    raise Exception("Invalid primary email address in test data.")

# Append the test data
append_to_sheet(test_data)
print("Test data appended to sheet.")

# Fetch the updated data
sheet_data = fetch_data_from_sheet()
if sheet_data:
    headers = sheet_data[0]
    rows = sheet_data[1:]
    df = pd.DataFrame(rows, columns=headers)

    # Calculate Health Score for all users
    df = calculate_health_score(df)

    # Assign ranks
    df = df.sort_values(by='HealthScore', ascending=False)
    df['Rank'] = range(1, len(df) + 1)

    # Get the new user's details
    new_user = df[df['Timestamp'] == timestamp].iloc[-1]
    rank = new_user['Rank']
    health_score = new_user['HealthScore']
    score_description = new_user['ScoreDescription']
    cash_flow_ratio = new_user['NormCashFlow']
    debt_to_income_ratio = new_user['DebtToIncomeRatio']
    debt_interest_burden = new_user['DebtInterestBurden']

    # Send email to the new user
    send_email(
        primary_email=new_user['Email'],
        fallback_email=new_user['AutoEmail'],
        first_name=new_user['FirstName'],
        last_name=new_user['LastName'],
        rank=rank,
        timestamp=timestamp,
        health_score=health_score,
        score_description=score_description,
        cash_flow_ratio=cash_flow_ratio,
        debt_to_income_ratio=debt_to_income_ratio,
        debt_interest_burden=debt_interest_burden
    )

    # Update the results sheet
    create_results_sheet()
    write_results_to_sheet(df)
    print("Test submission processed successfully.")
    display(df[['Email', 'HealthScore', 'Rank']])
else:
    print("Failed to fetch updated data after submission.")